# Playground for Visualising Data in KITTI

Notes for each size and variables:

seq_len = 11 (For LSTM)
imgs.shape = (16, 11, 3, 256, 512)
imus.shape = (16, 101, 6)


In [11]:
from pathlib import Path
import scipy.io as sio
import os
os.chdir('/home/marco/Documents/NeuralCDE-VIO')


1. Within function `make_dataset()`, it downloads imus data with shape (observations, 6)

In [6]:
root = Path('/mnt/data0/marco/Visual-Selective-VIO/data')
imus = sio.loadmat(root/'imus/{}.mat'.format('00'))
print(imus)
print(imus['imu_data_interp'].shape)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Jun  8 12:49:06 2021', '__version__': '1.0', '__globals__': [], 'imu_data_interp': array([[ 1.50400292e+00,  2.93219112e-01,  9.87464000e+00,
        -9.84725158e-04,  2.56165963e-02, -3.63842585e-04],
       [ 1.49926763e+00,  2.95990398e-01,  9.87622172e+00,
        -9.34152863e-04,  2.54884553e-02, -2.27065589e-04],
       [ 1.49453234e+00,  2.98761684e-01,  9.87780345e+00,
        -8.83580569e-04,  2.53603142e-02, -9.02885929e-05],
       ...,
       [-1.98989915e-01,  4.97848027e-01,  1.01454150e+01,
         2.52250288e-02, -3.79217315e-02, -6.68370680e-03],
       [-1.80242558e-01,  5.51190003e-01,  1.01775700e+01,
         2.14899408e-02, -3.92741115e-02, -6.53726925e-03],
       [-8.41972307e-02,  5.11396567e-01,  1.02197699e+01,
         1.68026044e-02, -4.04377488e-02, -6.10780227e-03]])}
(45401, 6)


2. Obtaining Pose Data: `make_dataset()`

In [13]:
import sys
print(sys.path)
from src.data.utils import rotationError, read_pose_from_text
root = Path('/mnt/data0/marco/Visual-Selective-VIO/data')
poses, poses_rel = read_pose_from_text(root/'poses/{}.txt'.format('00'))
print(poses.shape)
print(poses_rel.shape)

['/home/marco/Documents/NeuralCDE-VIO/notebooks/exploratory', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/marco/Documents/NeuralCDE-VIO/venv/lib/python3.10/site-packages']
(4541, 4, 4)
(4540, 6)


Digging deeper into `read_pose_from_text()`, we found that the poses information are stored as a NX12 table in poses.txt, where each row represent 1 measurement.

We use the following functions in utils to construct 4X4 homogenous matrix for pose representation.
In computer vision and robotics, a 4x4 matrix is commonly used to represent a 3D homogeneous transformation. A homogeneous transformation matrix allows us to represent translation, rotation, and scale in a unified way. The 4x4 matrix is convenient for representing 3D transformations because it can handle translations as well as rotations and scaling.

The 12 parameters you see in the code (presumably from a line in the KITTI odometry dataset) are used to construct a 3x4 matrix, where the first 3 columns represent the rotation and scaling part, and the last column represents the translation part.

For example, the 3x4 matrix might look like this:

\begin{bmatrix} 
    r_{11} & r_{12} & r_{13} & t_1 \\
    r_{21} & r_{22} & r_{23} & t_2 \\
    r_{31} & r_{32} & r_{33} & t_3 
\end{bmatrix} 

Here:
- \( r_{ij} \) represents the elements of the rotation and scaling submatrix.
- \( t_1, t_2, t_3 \) represent the translation vector.

To make it a 4x4 homogeneous transformation matrix, an extra row \([0, 0, 0, 1]\) is appended at the bottom:

\begin{bmatrix} 
    r_{11} & r_{12} & r_{13} & t_1 \\
    r_{21} & r_{22} & r_{23} & t_2 \\
    r_{31} & r_{32} & r_{33} & t_3 \\
    0 & 0 & 0 & 1 
\end{bmatrix} 

This homogeneous transformation matrix can be applied to 3D points using matrix multiplication, allowing for efficient transformations of 3D coordinates between different coordinate frames. It is a standard representation widely used in computer vision and robotics.


In [23]:
import numpy as np
import math


_EPS = np.finfo(float).eps * 4.0

# With the 3x4 transformation matrix, we add 1 row to the bottom, making it a 
# 4x4 homogeneous Matrix
def read_pose(line):
    '''
    Reading 4x4 pose matrix from .txt files
    input: a line of 12 parameters
    output: 4x4 numpy matrix
    '''
    values= np.reshape(np.array([float(value) for value in line.split(' ')]), (3, 4))
    Rt = np.concatenate((values, np.array([[0, 0, 0, 1]])), 0)
    return Rt
   
# Poses information is stored as a N x 12 table, where N is the number of
# frames of this sequence. Row i represents the i'th pose of the left camera
# coordinate system (i.e., z pointing forwards) via a 3x4 transformation
# matrix.

def read_pose_from_text(path):
    with open(path) as f:
        lines = [line.split('\n')[0] for line in f.readlines()]
        poses_rel, poses_abs = [], []
        values_p = read_pose(lines[0])
        poses_abs.append(values_p)            
        for i in range(1):
            values = read_pose(lines[i])
            poses_rel.append(get_relative_pose_6DoF(values_p, values)) 
            values_p = values.copy()
            poses_abs.append(values) 
        poses_abs = np.array(poses_abs)
        poses_rel = np.array(poses_rel)
        
    return poses_abs, poses_rel

def get_relative_pose(Rt1, Rt2):
    '''
    Calculate the relative 4x4 pose matrix between two pose matrices
    '''
    Rt1_inv = np.linalg.inv(Rt1)
    Rt_rel = Rt1_inv @ Rt2
    return Rt_rel

def get_relative_pose_6DoF(Rt1, Rt2):
    '''
    Calculate the relative rotation and translation from two consecutive pose matrices 
    '''
    
    # Calculate the relative transformation Rt_rel
    Rt_rel = get_relative_pose(Rt1, Rt2)

    R_rel = Rt_rel[:3, :3]
    t_rel = Rt_rel[:3, 3]

    # Extract the Eular angle from the relative rotation matrix
    x, y, z = euler_from_matrix(R_rel)
    theta = [x, y, z]

    pose_rel = np.concatenate((theta, t_rel))
    return pose_rel

def euler_from_matrix(matrix):
    '''
    Extract the eular angle from a rotation matrix
    '''
    M = np.array(matrix, dtype=np.float64, copy=False)[:3, :3]
    cy = math.sqrt(M[0, 0] * M[0, 0] + M[1, 0] * M[1, 0])
    ay = math.atan2(-M[2, 0], cy)
    if ay < -math.pi / 2 + _EPS and ay > -math.pi / 2 - _EPS:  # pitch = -90 deg
        ax = 0
        az = math.atan2(-M[1, 2], -M[0, 2])
    elif ay < math.pi / 2 + _EPS and ay > math.pi / 2 - _EPS:
        ax = 0
        az = math.atan2(M[1, 2], M[0, 2])
    else:
        ax = math.atan2(M[2, 1], M[2, 2])
        az = math.atan2(M[1, 0], M[0, 0])
    return np.array([ax, ay, az])

In [22]:
poses, poses_rel = read_pose_from_text(root/'poses/{}.txt'.format('00'))
print(poses)
print(poses.shape)
print(poses_rel)
print(poses_rel.shape)

[[[ 1.000000e+00  9.043680e-12  2.326809e-11  5.551115e-17]
  [ 9.043683e-12  1.000000e+00  2.392370e-10  3.330669e-16]
  [ 2.326810e-11  2.392370e-10  9.999999e-01 -4.440892e-16]
  [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]

 [[ 1.000000e+00  9.043680e-12  2.326809e-11  5.551115e-17]
  [ 9.043683e-12  1.000000e+00  2.392370e-10  3.330669e-16]
  [ 2.326810e-11  2.392370e-10  9.999999e-01 -4.440892e-16]
  [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]]
(2, 4, 4)
[[-2.06900257e-26 -2.47439078e-28  9.95909557e-29  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
(1, 6)


From the results above, we see that there are 2 4x4 matrix, and the difference of the two gives the 1x6 vector.

Understanding IMU Timestamps stored in KITTI Dataset

In [2]:
timestamp_dir = "/mnt/data0/marco/Visual-Selective-VIO/data/raw/2011_09_26/2011_09_26_drive_0001_sync/oxts/timestamps.txt"
image_dir = "/mnt/data0/marco/Visual-Selective-VIO/data/raw/2011_09_26/2011_09_26_drive_0001_sync/image_02/timestamps.txt"
def count_lines_in_file(file_path):
    """Count the number of lines in a text file.
    
    Args:
        file_path (str): The path to the text file.
        
    Returns:
        int: The number of lines in the file.
    """
    line_count = 0
    with open(file_path, 'r') as file:
        for line in file:
            line_count += 1
    return line_count

print(count_lines_in_file(timestamp_dir))
print(count_lines_in_file(image_dir))

108